In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os
from os.path import join
import re
import math
import pickle as pkl
import time
import numpy as np
import pandas as pd
from easydict import EasyDict as edict
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
from neuro_data_analysis.mat_data_translate_lib import h5_to_dict_simplify, print_hdf5_info
# set the pandas display options width
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)


mat_root = r"S:\Data-Ephys-MAT"
pkl_root = r"S:\Data-Ephys-PKL"
exp_record_pathdict = {"Alfa": r"S:\Exp_Record_Alfa.xlsx", 
                       "Beto": r"S:\ExpSpecTable_Augment.xlsx",
                       "Caos": r"S:\Exp_Record_Caos.xlsx",
                       "Diablito": r"S:\Exp_Record_Diablito.xlsx"}

### Find the older experiments

In [9]:
ExpRecord_All = pd.concat([pd.read_excel(exp_record_pathdict[Animal]).assign(Animal=Animal) for Animal in ("Caos", "Diablito", "Alfa", "Beto")])
exp_mask = ExpRecord_All.Exp_collection.str.contains('BigGAN_Hessian', na=False) & \
            ExpRecord_All.expControlFN.str.contains('selectivity', na=False) & \
                ExpRecord_All.Expi.notna()
ExpRecord_Hessian_All = ExpRecord_All.loc[exp_mask, :]
print("Hessian", ExpRecord_Hessian_All.shape)
ExpRecord_Hessian_All.groupby("Animal").count()

Hessian (55, 12)


,ChanQual,Exp_collection,Expi,WebCam,comments,ephysFN,expControlFN,pref_chan,pref_unit,stim_size,stimuli
Animal,,,,,,,,,,,
Alfa,6,29,29,0,29,29,29,29,28,29,29
Beto,0,14,14,0,14,14,14,14,14,14,14
Caos,0,7,7,0,7,7,7,7,7,7,7
Diablito,0,5,5,0,5,5,5,5,5,5,5


In [10]:
ExpRecord_All = pd.concat([pd.read_excel(exp_record_pathdict[Animal]).assign(Animal=Animal) for Animal in ("Caos", "Diablito", "Alfa", "Beto")])
exp_mask = ExpRecord_All.Exp_collection.str.contains('BigGAN_Hessian', na=False) & \
            ExpRecord_All.Exp_collection.str.contains('BigGAN_FC6', na=False) & \
            ExpRecord_All.expControlFN.str.contains('generate', na=False) & \
            ExpRecord_All.Expi.notna()
ExpRecord_Evolution_All = ExpRecord_All.loc[exp_mask, :]          
print("Evolution", ExpRecord_Evolution_All.shape)
ExpRecord_Evolution_All.groupby("Animal").count()

Evolution (52, 12)


,ChanQual,Exp_collection,Expi,WebCam,comments,ephysFN,expControlFN,pref_chan,pref_unit,stim_size,stimuli
Animal,,,,,,,,,,,
Alfa,6,26,26,0,26,26,26,26,25,26,26
Beto,0,14,14,0,14,14,14,14,14,14,14
Caos,0,7,7,0,7,7,7,7,7,7,7
Diablito,0,5,5,0,5,5,5,5,5,5,5


### Backup and sync the files 

In [11]:
def check_mat_file_existence(ExpRecord_All, 
                             local_mat_root=r"S:\Data-Ephys-MAT", 
                             network_mat_root=r"N:\Data-Ephys-MAT",
                             verbose=False):
    file_df = []
    for _, exprow in ExpRecord_All.iterrows():
        # check if the mat file exists
        net_raw_mat_path = join(network_mat_root, f"{exprow.ephysFN}.mat")
        net_formatted_mat_path = join(network_mat_root, f"{exprow.ephysFN}_formatted.mat")
        local_raw_mat_path = join(local_mat_root, f"{exprow.ephysFN}.mat")
        local_formatted_mat_path = join(local_mat_root, f"{exprow.ephysFN}_formatted.mat")
        exist_net_raw = os.path.exists(net_raw_mat_path)
        exist_net_formatted = os.path.exists(net_formatted_mat_path)
        exist_local_raw = os.path.exists(local_raw_mat_path)
        exist_local_formatted = os.path.exists(local_formatted_mat_path)
        # check the existence of the mat file, summarize the result in one line
        mat_exist_str = f"Net: Raw {exist_net_raw}, Formatted {exist_net_formatted}, Local: Raw {exist_local_raw}, Formatted {exist_local_formatted}"
        if verbose:
            print(exprow.ephysFN,exprow.Expi)   
            print(mat_exist_str)
        file_df.append({"ephysFN": exprow.ephysFN, "exist_net_raw": exist_net_raw, "exist_net_formatted": exist_net_formatted, 
                                "exist_local_raw": exist_local_raw, "exist_local_formatted": exist_local_formatted})
        # TODO: 
        # Policy: 
        #   if local exists any, copy the local raw / formatted file to the network folder
        #   if network exists formatted, copy the network formatted file to the local folder 
        #   if network exists raw, copy the network raw file to the local folder 
        
    file_df = pd.DataFrame(file_df)
    return file_df


In [12]:
file_df_Evolution = check_mat_file_existence(ExpRecord_Evolution_All, verbose=False)
file_df_Hessian = check_mat_file_existence(ExpRecord_Hessian_All, verbose=False)
assert file_df_Evolution.exist_local_formatted.all(), "Not all formatted files exist in the local folder, Evolution"
assert file_df_Hessian.exist_local_formatted.all(), "Not all formatted files exist in the local folder, Hessian"

In [14]:
from neuro_data_analysis.mat_data_translate_lib import batch_process_ephys
local_mat_root = r"S:\Data-Ephys-MAT"
batch_process_ephys(ExpRecord_Hessian_All.ephysFN.values, mat_root=local_mat_root, pkl_root=pkl_root, recompute=False)
batch_process_ephys(ExpRecord_Evolution_All.ephysFN.values, mat_root=local_mat_root, pkl_root=pkl_root, recompute=False)

PKL file already exists for Caos-12022024-003
PKL file already exists for Caos-12022024-006
PKL file already exists for Caos-12042024-003
PKL file already exists for Caos-12042024-007
PKL file already exists for Caos-12092024-004
PKL file already exists for Caos-12102024-003
PKL file already exists for Caos-12102024-006
PKL file already exists for Diablito-03122024-003
PKL file already exists for Diablito-11122024-003
PKL file already exists for Diablito-11122024-005
PKL file already exists for Diablito-12122024-004
PKL file already exists for Diablito-12122024-006
PKL file already exists for Alfa-04082020-003
PKL file already exists for Alfa-06082020-004
PKL file already exists for Alfa-10082020-004
PKL file already exists for Alfa-12082020-004
PKL file already exists for Alfa-14082020-003
PKL file already exists for Alfa-25082020-007
PKL file already exists for Alfa-27082020-003
PKL file already exists for Alfa-01092020-004
PKL file already exists for Alfa-02092020-004
PKL file alrea

In [41]:
local_mat_root = r"S:\Data-Ephys-MAT"
network_mat_root = r"N:\Data-Ephys-MAT"
file_df = []
for _, exprow in ExpRecord_Evolution_All.iterrows():
    # print(exprow.ephysFN,exprow.Expi)
    # check if the mat file exists
    net_raw_mat_path = join(network_mat_root, f"{exprow.ephysFN}.mat")
    net_formatted_mat_path = join(network_mat_root, f"{exprow.ephysFN}_formatted.mat")
    local_raw_mat_path = join(local_mat_root, f"{exprow.ephysFN}.mat")
    local_formatted_mat_path = join(local_mat_root, f"{exprow.ephysFN}_formatted.mat")
    exist_net_raw = os.path.exists(net_raw_mat_path)
    exist_net_formatted = os.path.exists(net_formatted_mat_path)
    exist_local_raw = os.path.exists(local_raw_mat_path)
    exist_local_formatted = os.path.exists(local_formatted_mat_path)
    # check the existence of the mat file, summarize the result in one line
    mat_exist_str = f"Net: Raw {exist_net_raw}, Formatted {exist_net_formatted}, Local: Raw {exist_local_raw}, Formatted {exist_local_formatted}"
    # print(mat_exist_str)
    # Policy: 
    #   if local exists any, copy the local raw / formatted file to the network folder
    #   if network exists formatted, copy the network formatted file to the local folder 
    #   if network exists raw, copy the network raw file to the local folder 
    
    file_df.append({"ephysFN": exprow.ephysFN, "exist_net_raw": exist_net_raw, "exist_net_formatted": exist_net_formatted, 
                              "exist_local_raw": exist_local_raw, "exist_local_formatted": exist_local_formatted})
file_df = pd.DataFrame(file_df)
file_df

,ephysFN,exist_net_raw,exist_net_formatted,exist_local_raw,exist_local_formatted
0,Caos-12022024-002,False,False,True,True
1,Caos-12022024-005,False,False,True,True
2,Caos-12042024-002,True,True,True,True
3,Caos-12042024-006,True,True,True,True
4,Caos-12092024-003,True,True,True,True
5,Caos-12102024-002,True,True,True,True
6,Caos-12102024-005,True,True,True,True
7,Diablito-03122024-002,True,True,True,True
8,Diablito-11122024-002,True,True,True,True
9,Diablito-11122024-004,True,True,True,True


In [42]:
local_mat_root = r"S:\Data-Ephys-MAT"
network_mat_root = r"N:\Data-Ephys-MAT"
file_df = []
for _, exprow in ExpRecord_Hessian_All.iterrows():
    # print(exprow.ephysFN,exprow.Expi)
    # check if the mat file exists
    net_raw_mat_path = join(network_mat_root, f"{exprow.ephysFN}.mat")
    net_formatted_mat_path = join(network_mat_root, f"{exprow.ephysFN}_formatted.mat")
    local_raw_mat_path = join(local_mat_root, f"{exprow.ephysFN}.mat")
    local_formatted_mat_path = join(local_mat_root, f"{exprow.ephysFN}_formatted.mat")
    exist_net_raw = os.path.exists(net_raw_mat_path)
    exist_net_formatted = os.path.exists(net_formatted_mat_path)
    exist_local_raw = os.path.exists(local_raw_mat_path)
    exist_local_formatted = os.path.exists(local_formatted_mat_path)
    # check the existence of the mat file, summarize the result in one line
    mat_exist_str = f"Net: Raw {exist_net_raw}, Formatted {exist_net_formatted}, Local: Raw {exist_local_raw}, Formatted {exist_local_formatted}"
    # print(mat_exist_str)
    # Policy: 
    #   if local exists any, copy the local raw / formatted file to the network folder
    #   if network exists formatted, copy the network formatted file to the local folder 
    #   if network exists raw, copy the network raw file to the local folder 
    
    file_df.append({"ephysFN": exprow.ephysFN, "exist_net_raw": exist_net_raw, "exist_net_formatted": exist_net_formatted, 
                              "exist_local_raw": exist_local_raw, "exist_local_formatted": exist_local_formatted})
file_df = pd.DataFrame(file_df)
file_df

,ephysFN,exist_net_raw,exist_net_formatted,exist_local_raw,exist_local_formatted
0,Caos-12022024-003,False,False,True,True
1,Caos-12022024-006,False,False,True,True
2,Caos-12042024-003,True,True,True,True
3,Caos-12042024-007,True,True,True,True
4,Caos-12092024-004,True,True,True,True
5,Caos-12102024-003,True,True,True,True
6,Caos-12102024-006,True,True,True,True
7,Diablito-03122024-003,True,True,True,True
8,Diablito-11122024-003,True,True,True,True
9,Diablito-11122024-005,True,True,True,True


`Alfa-14052021-004`	 this one does not exist in the network let's process it.

In [45]:
from neuro_data_analysis.mat_data_translate_lib import batch_process_ephys
batch_process_ephys(ExpRecord_Hessian_All.ephysFN.values, mat_root=network_mat_root, pkl_root=pkl_root, recompute=False)
batch_process_ephys(ExpRecord_Evolution_All.ephysFN.values, mat_root=network_mat_root, pkl_root=pkl_root, recompute=False)

PKL file already exists for Caos-12022024-003
PKL file already exists for Caos-12022024-006
PKL file already exists for Caos-12042024-003
PKL file already exists for Caos-12042024-007
PKL file already exists for Caos-12092024-004
PKL file already exists for Caos-12102024-003
PKL file already exists for Caos-12102024-006
PKL file already exists for Diablito-03122024-003
PKL file already exists for Diablito-11122024-003
PKL file already exists for Diablito-11122024-005
PKL file already exists for Diablito-12122024-004
PKL file already exists for Diablito-12122024-006
PKL file already exists for Alfa-04082020-003
PKL file already exists for Alfa-06082020-004
PKL file already exists for Alfa-10082020-004
PKL file already exists for Alfa-12082020-004
PKL file already exists for Alfa-14082020-003
PKL file already exists for Alfa-25082020-007
PKL file already exists for Alfa-27082020-003
PKL file already exists for Alfa-01092020-004
PKL file already exists for Alfa-02092020-004
PKL file alrea

In [64]:
batch_process_ephys(["Alfa-14052021-004"], mat_root=local_mat_root, pkl_root=pkl_root, recompute=False)

Processing Alfa-14052021-004...
Loading Alfa-14052021-004 mat file
Time taken: 0.00 seconds for loading mat file
Loading TrialRecord	Warning: No MATLAB class specified for <HDF5 dataset "Angle": shape (7, 1), type "|O"> at path /Trials/TrialRecord/CurrentConditionStimulusInfo/Angle object
done
Loading XY	done
Loading block	done
Loading event03	done
Loading event10	done
Loading eventMarkers	done
Loading eyePupil	done
Loading eyeXY	done
Loading imageInTrial	done
Loading imageName	done
Loading imageOFFtime	done
Loading imageONtime	done
Loading trialNum	done
Loading trialStart	done
Loading width	done
Loading words	done
Time taken: 8.14 seconds for loading trials
Saved Alfa-14052021-004 to S:\Data-Ephys-PKL\Alfa-14052021-004.pkl



In [65]:
from neuro_data_analysis.mat_data_translate_lib import batch_process_ephys
batch_process_ephys(ExpRecord_Hessian_All.ephysFN.values, mat_root=local_mat_root, pkl_root=pkl_root, recompute=False)
batch_process_ephys(ExpRecord_Evolution_All.ephysFN.values, mat_root=local_mat_root, pkl_root=pkl_root, recompute=False)

PKL file already exists for Caos-12022024-003
PKL file already exists for Caos-12022024-006
PKL file already exists for Caos-12042024-003
PKL file already exists for Caos-12042024-007
PKL file already exists for Caos-12092024-004
PKL file already exists for Caos-12102024-003
PKL file already exists for Caos-12102024-006
PKL file already exists for Diablito-03122024-003
PKL file already exists for Diablito-11122024-003
PKL file already exists for Diablito-11122024-005
PKL file already exists for Diablito-12122024-004
PKL file already exists for Diablito-12122024-006
PKL file already exists for Alfa-04082020-003
PKL file already exists for Alfa-06082020-004
PKL file already exists for Alfa-10082020-004
PKL file already exists for Alfa-12082020-004
PKL file already exists for Alfa-14082020-003
PKL file already exists for Alfa-25082020-007
PKL file already exists for Alfa-27082020-003
PKL file already exists for Alfa-01092020-004
PKL file already exists for Alfa-02092020-004
PKL file alrea

### Go through meta data of the evolution experiments

In [68]:
for _, exprow in ExpRecord_Evolution_All.iterrows():
    print(exprow.ephysFN, "Exp", int(exprow.Expi))
    data = pkl.load(open(join(pkl_root, f"{exprow.ephysFN}.pkl"), "rb"))
    rasters = data["rasters"]
    meta = data["meta"]
    Trials = data["Trials"]
    imageName = np.squeeze(Trials.imageName)
    stimuli_dir = exprow.stimuli
    prefchan = np.squeeze(Trials.TrialRecord.User.prefChan)
    prefunit = np.squeeze(Trials.TrialRecord.User.unit_perChan)
    imgpos = np.squeeze(Trials.TrialRecord.User.evoConfiguration[1])
    imgwidth = np.squeeze(Trials.TrialRecord.User.evoConfiguration[2])
    print(prefchan[0], prefunit[0], imgpos[0].T, imgwidth[0])

Caos-12022024-002 Exp 1
91.0 2.0 [[0. 0.]] [[2.]]
Caos-12022024-005 Exp 2
94.0 1.0 [[0. 0.]] [[2.]]
Caos-12042024-002 Exp 3
71.0 2.0 [[0. 0.]] [[2.]]
Caos-12042024-006 Exp 4
72.0 1.0 [[0. 0.]] [[2.]]
Caos-12092024-003 Exp 5
92.0 2.0 [[0. 0.]] [[2.]]
Caos-12102024-002 Exp 6
68.0 2.0 [[0. 0.]] [[2.]]
Caos-12102024-005 Exp 7
69.0 1.0 [[0. 0.]] [[2.]]
Diablito-03122024-002 Exp 1
56.0 1.0 [[-1.  0.]] [[4.]]
Diablito-11122024-002 Exp 2
42.0 2.0 [[-1. -1.]] [[4.]]
Diablito-11122024-004 Exp 3
34.0 2.0 [[-2.  0.]] [[4.]]
Diablito-12122024-003 Exp 4
64.0 2.0 [[-1. -1.]] [[6.]]
Diablito-12122024-005 Exp 5
50.0 1.0 [[0. 0.]] [[4.]]
Alfa-04082020-002 Exp 1
22.0 1.0 [[ 0.  -2.5]] [[3.]]
Alfa-06082020-002 Exp 2
6.0 1.0 [[-0.3 -3. ]] [[3.]]
Alfa-12082020-003 Exp 3
29.0 1.0 [[ 0. -2.]] [[4.]]
Alfa-14082020-002 Exp 4
51.0 1.0 [[-1.8 -1.8]] [[4.]]
Alfa-25082020-006 Exp 5
59.0 1.0 [[-2.5 -2.5]] [[3.]]
Alfa-27082020-002 Exp 6
7.0 1.0 [[-1.5 -1.5]] [[3.]]
Alfa-02092020-003 Exp 8
64.0 1.0 [[ 0. -1.]] [[3.]]


### Automatic checking of meta and fill in missing information

In [79]:
ExpRecord_Evolution_All.to_csv(r"ExpRecord_BigGAN_Hessian_Evol_ABCD.csv", index=False)

In [19]:
def safe_update_column(table_original, row_in_original, row, column):
    if pd.notna(row_in_original[column]).all():
        assert (row_in_original[column] == row[column]).all(), f"{column} mismatch: {row_in_original[column]} != {row[column]}"
    else:
        table_original.loc[row_in_original.index, column] = row[column]


def update_table_with_assertion(table, row_index, column_name, new_value):
    if pd.notna(table.loc[row_index, column_name]):
        assert table.loc[row_index, column_name] == new_value, f"{column_name} mismatch: {table.loc[row_index, column_name]} != {new_value}"
    else:
        table.loc[row_index, column_name] = new_value

In [21]:
ExpRecord_Evolution_All = ExpRecord_Evolution_All.reset_index(drop=True)

for rowi, exprow in ExpRecord_Evolution_All.iterrows():
    print(exprow.ephysFN, exprow.Animal, "Exp", int(exprow.Expi))
    data = pkl.load(open(join(pkl_root, f"{exprow.ephysFN}.pkl"), "rb"))
    rasters = data["rasters"]
    meta = data["meta"]
    Trials = data["Trials"]
    stimuli_dir = exprow.stimuli
    prefchan = int(np.squeeze(Trials.TrialRecord.User.prefChan)[0])
    prefunit = int(np.squeeze(Trials.TrialRecord.User.unit_perChan)[0])
    imgpos = np.squeeze(Trials.TrialRecord.User.evoConfiguration[1])[0].T
    imgwidth = float(np.squeeze(Trials.TrialRecord.User.evoConfiguration[2])[0])
    print(prefchan, prefunit, imgpos, imgwidth)
    update_table_with_assertion(ExpRecord_Evolution_All, rowi, "pref_chan", prefchan)
    update_table_with_assertion(ExpRecord_Evolution_All, rowi, "pref_unit", prefunit)
    update_table_with_assertion(ExpRecord_Evolution_All, rowi, "stim_size", imgwidth)

ExpRecord_Evolution_All.to_csv(r"ExpRecord_BigGAN_Hessian_Evol_ABCD_w_meta.csv", index=False)

Caos-12022024-002 Caos Exp 1
91 2 [[0. 0.]] 2.0
Caos-12022024-005 Caos Exp 2
94 1 [[0. 0.]] 2.0
Caos-12042024-002 Caos Exp 3
71 2 [[0. 0.]] 2.0
Caos-12042024-006 Caos Exp 4
72 1 [[0. 0.]] 2.0
Caos-12092024-003 Caos Exp 5
92 2 [[0. 0.]] 2.0
Caos-12102024-002 Caos Exp 6
68 2 [[0. 0.]] 2.0
Caos-12102024-005 Caos Exp 7
69 1 [[0. 0.]] 2.0
Diablito-03122024-002 Diablito Exp 1
56 1 [[-1.  0.]] 4.0
Diablito-11122024-002 Diablito Exp 2
42 2 [[-1. -1.]] 4.0
Diablito-11122024-004 Diablito Exp 3
34 2 [[-2.  0.]] 4.0
Diablito-12122024-003 Diablito Exp 4
64 2 [[-1. -1.]] 6.0
Diablito-12122024-005 Diablito Exp 5
50 1 [[0. 0.]] 4.0
Alfa-04082020-002 Alfa Exp 1
22 1 [[ 0.  -2.5]] 3.0
Alfa-06082020-002 Alfa Exp 2
6 1 [[-0.3 -3. ]] 3.0
Alfa-12082020-003 Alfa Exp 3
29 1 [[ 0. -2.]] 4.0
Alfa-14082020-002 Alfa Exp 4
51 1 [[-1.8 -1.8]] 4.0
Alfa-25082020-006 Alfa Exp 5
59 1 [[-2.5 -2.5]] 3.0
Alfa-27082020-002 Alfa Exp 6
7 1 [[-1.5 -1.5]] 3.0
Alfa-01092020-003 Alfa Exp 7
52 1 [[0. 0.]] 3.0
Alfa-02092020-003 Al

In [22]:
ExpRecord_Hessian_All.to_csv(r"ExpRecord_BigGAN_Hessian_tuning_ABCD.csv", index=False)

In [23]:
ExpRecord_Hessian_All = ExpRecord_Hessian_All.reset_index(drop=True)
ExpRecord_Evolution_All = ExpRecord_Evolution_All.reset_index(drop=True)
for rowi, exprow in ExpRecord_Evolution_All.iterrows():
    print(exprow.ephysFN, exprow.Animal, "Exp", int(exprow.Expi))
    data = pkl.load(open(join(pkl_root, f"{exprow.ephysFN}.pkl"), "rb"))
    rasters = data["rasters"]
    meta = data["meta"]
    Trials = data["Trials"]
    stimuli_dir = exprow.stimuli
    prefchan = int(np.squeeze(Trials.TrialRecord.User.prefChan)[0])
    prefunit = int(np.squeeze(Trials.TrialRecord.User.unit_perChan)[0])
    imgpos = np.squeeze(Trials.TrialRecord.User.evoConfiguration[1])[0].T
    imgwidth = float(np.squeeze(Trials.TrialRecord.User.evoConfiguration[2])[0])
    print(prefchan, prefunit, imgpos, imgwidth)
    # find the corresponding row in ExpRecord_Hessian_All
    hessian_exprow = ExpRecord_Hessian_All.query(f"Animal == '{exprow.Animal}' and Expi == {exprow.Expi}")
    safe_update_column(ExpRecord_Hessian_All, hessian_exprow, exprow, "pref_chan")
    safe_update_column(ExpRecord_Hessian_All, hessian_exprow, exprow, "pref_unit")
    safe_update_column(ExpRecord_Hessian_All, hessian_exprow, exprow, "stim_size")
    
# ExpRecord_Evolution_All.to_csv(r"ExpRecord_BigGAN_Hessian_Evol_ABCD_w_meta.csv", index=False)

Caos-12022024-002 Caos Exp 1
91 2 [[0. 0.]] 2.0
Caos-12022024-005 Caos Exp 2
94 1 [[0. 0.]] 2.0
Caos-12042024-002 Caos Exp 3
71 2 [[0. 0.]] 2.0
Caos-12042024-006 Caos Exp 4
72 1 [[0. 0.]] 2.0
Caos-12092024-003 Caos Exp 5
92 2 [[0. 0.]] 2.0
Caos-12102024-002 Caos Exp 6
68 2 [[0. 0.]] 2.0
Caos-12102024-005 Caos Exp 7
69 1 [[0. 0.]] 2.0
Diablito-03122024-002 Diablito Exp 1
56 1 [[-1.  0.]] 4.0
Diablito-11122024-002 Diablito Exp 2
42 2 [[-1. -1.]] 4.0
Diablito-11122024-004 Diablito Exp 3
34 2 [[-2.  0.]] 4.0
Diablito-12122024-003 Diablito Exp 4
64 2 [[-1. -1.]] 6.0
Diablito-12122024-005 Diablito Exp 5
50 1 [[0. 0.]] 4.0
Alfa-04082020-002 Alfa Exp 1
22 1 [[ 0.  -2.5]] 3.0
Alfa-06082020-002 Alfa Exp 2
6 1 [[-0.3 -3. ]] 3.0
Alfa-12082020-003 Alfa Exp 3
29 1 [[ 0. -2.]] 4.0
Alfa-14082020-002 Alfa Exp 4
51 1 [[-1.8 -1.8]] 4.0
Alfa-25082020-006 Alfa Exp 5
59 1 [[-2.5 -2.5]] 3.0
Alfa-27082020-002 Alfa Exp 6
7 1 [[-1.5 -1.5]] 3.0
Alfa-01092020-003 Alfa Exp 7
52 1 [[0. 0.]] 3.0
Alfa-02092020-003 Al

In [25]:
ExpRecord_Hessian_All.to_csv(r"ExpRecord_BigGAN_Hessian_tuning_ABCD_w_meta.csv", index=False)

### Write back the meta information to the original excel file in a safe way


First backup the original excel file as csv etc. 

In [27]:
exp_record_pathdict = {"Alfa": r"S:\Exp_Record_Alfa.xlsx", 
                       "Beto": r"S:\ExpSpecTable_Augment.xlsx",
                       "Caos": r"S:\Exp_Record_Caos.xlsx",
                       "Diablito": r"S:\Exp_Record_Diablito.xlsx"}
ExpRecord_dict = {Animal: pd.read_excel(exp_record_pathdict[Animal]) for Animal in exp_record_pathdict.keys()}
# save each ExpRecord_dict to a csv file
for Animal, exp_original in ExpRecord_dict.items():
    exp_original.to_csv(exp_record_pathdict[Animal].replace(".xlsx", ".csv"), index=False)
    exp_original.to_excel(exp_record_pathdict[Animal].replace(".xlsx", "BackupCopy.xlsx"), index=False)

For the new exp record 

In [104]:
exp_record_pathdict = {"Alfa": r"S:\Exp_Record_Alfa.xlsx", 
                       "Beto": r"S:\ExpSpecTable_Augment.xlsx",
                       "Caos": r"S:\Exp_Record_Caos.xlsx",
                       "Diablito": r"S:\Exp_Record_Diablito.xlsx"}

ExpRecord_dict = {Animal: pd.read_excel(exp_record_pathdict[Animal]) for Animal in exp_record_pathdict.keys()}

for _, exprow in pd.concat([ExpRecord_Hessian_All, ExpRecord_Evolution_All]).iterrows():
    print(exprow.ephysFN, exprow.Animal, "Exp", int(exprow.Expi))
    exp_original = ExpRecord_dict[exprow.Animal]
    # find the corresponding row in the original excel file 
    exp_in_original = exp_original.query(f"ephysFN == '{exprow.ephysFN}'")
    # print(exp_in_original)
    # write the meta information to the original excel file. if the information is already there, check if it is the same
    if pd.notna(exp_in_original.pref_chan).all():
        assert (exp_in_original.pref_chan == exprow.pref_chan).all(), f"Pref chan mismatch: {exp_in_original.pref_chan} != {exprow.pref_chan}"
    else:
        exp_original.loc[exp_in_original.index, "pref_chan"] = exprow.pref_chan
    
    if pd.notna(exp_in_original.pref_unit).all():
        assert (exp_in_original.pref_unit == exprow.pref_unit).all(), f"Pref unit mismatch: {exp_in_original.pref_unit} != {exprow.pref_unit}"
    else:
        exp_original.loc[exp_in_original.index, "pref_unit"] = exprow.pref_unit
    
    if pd.notna(exp_in_original.stim_size).all():
        assert (exp_in_original.stim_size == exprow.stim_size).all(), f"Stim size mismatch: {exp_in_original.stim_size} != {exprow.stim_size}"
    else:
        exp_original.loc[exp_in_original.index, "stim_size"] = exprow.stim_size


for Animal, exp_original in ExpRecord_dict.items():
    exp_original.to_excel(exp_record_pathdict[Animal], index=False)
    exp_original.to_csv(exp_record_pathdict[Animal].replace(".xlsx", ".csv"), index=False)


Caos-12022024-003 Caos Exp 1
Caos-12022024-006 Caos Exp 2
Caos-12042024-003 Caos Exp 3
Caos-12042024-007 Caos Exp 4
Caos-12092024-004 Caos Exp 5
Caos-12102024-003 Caos Exp 6
Caos-12102024-006 Caos Exp 7
Diablito-03122024-003 Diablito Exp 1
Diablito-11122024-003 Diablito Exp 2
Diablito-11122024-005 Diablito Exp 3
Diablito-12122024-004 Diablito Exp 4
Diablito-12122024-006 Diablito Exp 5
Alfa-04082020-003 Alfa Exp 1
Alfa-06082020-004 Alfa Exp 2
Alfa-10082020-004 Alfa Exp 2
Alfa-12082020-004 Alfa Exp 3
Alfa-14082020-003 Alfa Exp 4
Alfa-25082020-007 Alfa Exp 5
Alfa-27082020-003 Alfa Exp 6
Alfa-01092020-004 Alfa Exp 7
Alfa-02092020-004 Alfa Exp 8
Alfa-02092020-006 Alfa Exp 9
Alfa-08092020-004 Alfa Exp 10
Alfa-11092020-003 Alfa Exp 11
Alfa-11092020-006 Alfa Exp 12
Alfa-16092020-003 Alfa Exp 13
Alfa-16092020-005 Alfa Exp 14
Alfa-27102020-007 Alfa Exp 15
Alfa-27102020-008 Alfa Exp 15
Alfa-27102020-009 Alfa Exp 15
Alfa-04112020-005 Alfa Exp 16
Alfa-06112020-006 Alfa Exp 17
Alfa-12012021-003 Alfa

A more modular version of the code. 

In [29]:
exp_record_pathdict = {
    "Alfa": r"S:\Exp_Record_Alfa.xlsx", 
    "Beto": r"S:\ExpSpecTable_Augment.xlsx",
    "Caos": r"S:\Exp_Record_Caos.xlsx",
    "Diablito": r"S:\Exp_Record_Diablito.xlsx"
}

ExpRecord_dict = {Animal: pd.read_excel(path) for Animal, path in exp_record_pathdict.items()}

for _, exprow in pd.concat([ExpRecord_Hessian_All, ExpRecord_Evolution_All]).iterrows():
    print(exprow.ephysFN, exprow.Animal, "Exp", int(exprow.Expi))
    table_original = ExpRecord_dict[exprow.Animal]
    exprow_in_original = table_original.query(f"ephysFN == '{exprow.ephysFN}'")
    if not exprow_in_original.empty:  # Ensure the row exists before updating
        for column in ["pref_chan", "pref_unit", "stim_size"]:
            safe_update_column(table_original, exprow_in_original, exprow, column)
    else:
        assert False, f"Row {exprow.ephysFN} not found in {exprow.Animal} table"

for Animal, exp_original in ExpRecord_dict.items():
    exp_original.to_excel(exp_record_pathdict[Animal], index=False)
    exp_original.to_csv(exp_record_pathdict[Animal].replace(".xlsx", ".csv"), index=False)

Caos-12022024-003 Caos Exp 1
Caos-12022024-006 Caos Exp 2
Caos-12042024-003 Caos Exp 3
Caos-12042024-007 Caos Exp 4
Caos-12092024-004 Caos Exp 5
Caos-12102024-003 Caos Exp 6
Caos-12102024-006 Caos Exp 7
Diablito-03122024-003 Diablito Exp 1
Diablito-11122024-003 Diablito Exp 2
Diablito-11122024-005 Diablito Exp 3
Diablito-12122024-004 Diablito Exp 4
Diablito-12122024-006 Diablito Exp 5
Alfa-04082020-003 Alfa Exp 1
Alfa-06082020-004 Alfa Exp 2
Alfa-10082020-004 Alfa Exp 2
Alfa-12082020-004 Alfa Exp 3
Alfa-14082020-003 Alfa Exp 4
Alfa-25082020-007 Alfa Exp 5
Alfa-27082020-003 Alfa Exp 6
Alfa-01092020-004 Alfa Exp 7
Alfa-02092020-004 Alfa Exp 8
Alfa-02092020-006 Alfa Exp 9
Alfa-08092020-004 Alfa Exp 10
Alfa-11092020-003 Alfa Exp 11
Alfa-11092020-006 Alfa Exp 12
Alfa-16092020-003 Alfa Exp 13
Alfa-16092020-005 Alfa Exp 14
Alfa-27102020-007 Alfa Exp 15
Alfa-27102020-008 Alfa Exp 15
Alfa-27102020-009 Alfa Exp 15
Alfa-04112020-005 Alfa Exp 16
Alfa-06112020-006 Alfa Exp 17
Alfa-12012021-003 Alfa

WE can use the `Spreadsheet Compare (Part of Office Professional)` to compare the two excel files. Making sure that the changes are correct. 

### Check the meta data in Evolution and Hessian experiment match each other

In [32]:
ExpRecord_Evolution_All = pd.read_csv(r"ExpRecord_BigGAN_Hessian_Evol_ABCD_w_meta.csv")
ExpRecord_Hessian_All = pd.read_csv(r"ExpRecord_BigGAN_Hessian_tuning_ABCD_w_meta.csv")

for _, exprow in ExpRecord_Evolution_All.iterrows():
    print(exprow.ephysFN, exprow.Animal, "Exp", int(exprow.Expi))
    hessian_exprow = ExpRecord_Hessian_All.query(f"Animal == '{exprow.Animal}' and Expi == {exprow.Expi}")
    assert (hessian_exprow.pref_chan == exprow.pref_chan).all(), f"Pref chan mismatch: {hessian_exprow.pref_chan} != {exprow.pref_chan}"
    assert (hessian_exprow.pref_unit == exprow.pref_unit).all(), f"Pref unit mismatch: {hessian_exprow.pref_unit} != {exprow.pref_unit}"
    assert (hessian_exprow.stim_size == exprow.stim_size).all(), f"Stim size mismatch: {hessian_exprow.stim_size} != {exprow.stim_size}"

for _, exprow in ExpRecord_Hessian_All.iterrows():
    print(exprow.ephysFN, exprow.Animal, "Exp", int(exprow.Expi))
    evolution_exprow = ExpRecord_Evolution_All.query(f"Animal == '{exprow.Animal}' and Expi == {exprow.Expi}")
    assert (evolution_exprow.pref_chan == exprow.pref_chan).all(), f"Pref chan mismatch: {evolution_exprow.pref_chan} != {exprow.pref_chan}"
    assert (evolution_exprow.pref_unit == exprow.pref_unit).all(), f"Pref unit mismatch: {evolution_exprow.pref_unit} != {exprow.pref_unit}"
    assert (evolution_exprow.stim_size == exprow.stim_size).all(), f"Stim size mismatch: {evolution_exprow.stim_size} != {exprow.stim_size}"


Caos-12022024-002 Caos Exp 1
Caos-12022024-005 Caos Exp 2
Caos-12042024-002 Caos Exp 3
Caos-12042024-006 Caos Exp 4
Caos-12092024-003 Caos Exp 5
Caos-12102024-002 Caos Exp 6
Caos-12102024-005 Caos Exp 7
Diablito-03122024-002 Diablito Exp 1
Diablito-11122024-002 Diablito Exp 2
Diablito-11122024-004 Diablito Exp 3
Diablito-12122024-003 Diablito Exp 4
Diablito-12122024-005 Diablito Exp 5
Alfa-04082020-002 Alfa Exp 1
Alfa-06082020-002 Alfa Exp 2
Alfa-12082020-003 Alfa Exp 3
Alfa-14082020-002 Alfa Exp 4
Alfa-25082020-006 Alfa Exp 5
Alfa-27082020-002 Alfa Exp 6
Alfa-01092020-003 Alfa Exp 7
Alfa-02092020-003 Alfa Exp 8
Alfa-02092020-005 Alfa Exp 9
Alfa-08092020-002 Alfa Exp 10
Alfa-11092020-002 Alfa Exp 11
Alfa-11092020-005 Alfa Exp 12
Alfa-16092020-002 Alfa Exp 13
Alfa-16092020-004 Alfa Exp 14
Alfa-27102020-004 Alfa Exp 15
Alfa-04112020-002 Alfa Exp 16
Alfa-06112020-003 Alfa Exp 17
Alfa-12012021-002 Alfa Exp 18
Alfa-13012021-002 Alfa Exp 19
Alfa-19022021-003 Alfa Exp 20
Alfa-25022021-002 Alf

Check the two tables have the same

In [54]:
tmp_Hessian = ExpRecord_Hessian_All[["Animal", "Expi", "pref_chan", "pref_unit", "stim_size"]].drop_duplicates().sort_values(by=["Animal", "Expi"]).reset_index(drop=True)
tmp_Evolution = ExpRecord_Evolution_All[["Animal", "Expi", "pref_chan", "pref_unit", "stim_size"]].drop_duplicates().sort_values(by=["Animal", "Expi"]).reset_index(drop=True)
# # find the difference between the two
tmp_Hessian.equals(tmp_Evolution)

True

### Use PL2 files to update the pkl files 

In [4]:
ExpRecord_Hessian_All = pd.read_csv(r"ExpRecord_BigGAN_Hessian_tuning_ABCD_w_meta.csv")
ExpRecord_Evolution_All = pd.read_csv(r"ExpRecord_BigGAN_Hessian_Evol_ABCD_w_meta.csv")


In [6]:
from tqdm.auto import tqdm


c:\Users\ponce\.conda\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from neuro_data_analysis.PL2_reading_lib import read_channel_unit_info, pl2_info


In [13]:
pl2_info(join(pkl_root, f"{exprow.ephysFN}.pl2"))

b'unable to open file'


0

In [28]:
exprow.ephysFN

'Alfa-08042021-002'

In [34]:
pd.concat([ExpRecord_Hessian_All, ExpRecord_Evolution_All]).query("ephysFN == 'Alfa-27082020-003'")

,ChanQual,Exp_collection,Expi,WebCam,comments,ephysFN,expControlFN,pref_chan,pref_unit,stim_size,stimuli,Animal
18,NaN,BigGAN_Hessian,6.0,NaN,Selectivity start at 1018. Hess iamges from CH...,Alfa-27082020-003,200827_Alfa_selectivity_basic,7.0,1.0,3.0,N:\Stimuli\2020-Hessian\alfa-200827-selectivity,Alfa


In [40]:
print(pd.concat([ExpRecord_Hessian_All, ExpRecord_Evolution_All]).query("ephysFN == 'Alfa-27082020-002'").iloc[0])


ChanQual                                                        NaN
Exp_collection                            BigGAN_FC6;BigGAN_Hessian
Expi                                                            6.0
WebCam                                                          NaN
comments          CH 7 (-1.5, -1.5) 3 1 CMAES hash 1/5. Fc6 vs b...
ephysFN                                           Alfa-27082020-002
expControlFN                         200827_Alfa_generate_BigGAN(1)
pref_chan                                                       7.0
pref_unit                                                       1.0
stim_size                                                       3.0
stimuli           N:\Stimuli\2020-BigGAN\2020-08-27-Alfa-01\2020...
Animal                                                         Alfa
Name: 17, dtype: object


In [41]:
print(pd.concat([ExpRecord_Hessian_All, ExpRecord_Evolution_All]).query("ephysFN == 'Alfa-27082020-002'").iloc[0].stimuli)


N:\Stimuli\2020-BigGAN\2020-08-27-Alfa-01\2020-08-27-09-41-28


In [33]:
PL2_roots = [r"S:\Data-Ephys-Raw", r"N:\Data-Ephys-Raw",
             r"\\standby.files.med.harvard.edu\hms\neurobio\ponce\collaborations\Data-Ephys-Raw\2020",
             r"\\standby.files.med.harvard.edu\hms\neurobio\ponce\collaborations\Data-Ephys-Raw\2021"]
for _, exprow in tqdm(pd.concat([ExpRecord_Hessian_All, ExpRecord_Evolution_All]).iterrows()):   
    PL2_path = None
    for root in PL2_roots:
        potential_path = join(root, f"{exprow.ephysFN}.pl2")
        if os.path.exists(potential_path):
            PL2_path = potential_path
            break
    if PL2_path is None:
        print(f"PL2 file does not exist in any of the provided roots: {exprow.ephysFN}.pl2")
        continue
    spike_channel_id, spike_unit_id, spike_total_cnt = read_channel_unit_info(PL2_path)
    pkldata = pkl.load(open(join(pkl_root, f"{exprow.ephysFN}.pkl"), "rb"))
    if "spikeID" in pkldata["meta"].keys():
        channel_id_pkl = pkldata["meta"]["spikeID"][0]
        length_match = len(channel_id_pkl) == len(spike_channel_id)
        exact_match = (channel_id_pkl.astype(int) == spike_channel_id)
        if not (length_match and exact_match.all()):
            print(f"{exprow.ephysFN} Spike ID mismatch: {channel_id_pkl} [len: {len(channel_id_pkl)}] != {spike_channel_id} [len: {len(spike_channel_id)}]")
    # else:
    #     pkldata["meta"]["spikeID"] = spike_channel_id[None, :]
        
    if "unitID" in pkldata["meta"].keys():
        unit_id_pkl = pkldata["meta"]["unitID"][0]
        length_match = len(unit_id_pkl) == len(spike_unit_id)
        exact_match = (unit_id_pkl.astype(int) == spike_unit_id)
        if not (length_match and exact_match.all()):
            print(f"{exprow.ephysFN} Unit ID mismatch: {unit_id_pkl} [len: {len(unit_id_pkl)}] != {spike_unit_id} [len: {len(spike_unit_id)}]")

0it [00:00, ?it/s]

18it [00:04,  3.95it/s]

PL2 file does not exist in any of the provided roots: Alfa-27082020-003.pl2


72it [00:18,  3.04it/s]

PL2 file does not exist in any of the provided roots: Alfa-27082020-002.pl2


...

92it [00:24,  2.90it/s]

Alfa-08042021-002 Spike ID mismatch: [ 1.  1.  2.  2.  3.  3.  4.  4.  5.  5.  6.  6.  7.  7.  8.  8.  9.  9.
 10. 10. 11. 11. 12. 12. 13. 13. 14. 14. 15. 15. 16. 17. 17. 18. 18. 19.
 19. 20. 20. 20. 21. 21. 22. 22. 22. 23. 23. 24. 24. 25. 25. 26. 26. 27.
 27. 28. 28. 29. 29. 30. 30. 30. 31. 31. 32. 32. 32. 33. 33. 33. 34. 34.
 35. 35. 36. 36. 37. 37. 38. 38. 39. 39. 39. 40. 40. 41. 41. 42. 42. 43.
 43. 43. 43. 44. 44. 45. 45. 46. 46. 47. 47. 48. 48. 49. 49. 50. 50. 51.
 51. 52. 52. 53. 53. 54. 54. 55. 55. 56. 56. 57. 57. 58. 58. 58. 59. 59.
 59. 60. 60. 61. 61. 62. 62. 63. 63. 64.] [len: 136] != [ 1  1  2  2  3  3  4  5  5  6  7  7  8  9  9 10 11 12 12 13 14 14 15 15
 16 17 18 19 19 20 20 20 21 22 22 22 23 24 24 25 26 27 28 29 30 30 31 32
 32 32 33 33 33 34 34 35 35 36 36 37 37 38 38 39 39 39 40 40 41 41 42 42
 43 43 43 44 44 45 45 46 46 47 47 48 48 49 49 50 50 51 51 52 52 53 53 54
 54 55 55 56 56 57 57 58 58 58 59 59 59 60 60 61 61 62 62 63 63 64] [len: 118]
Alfa-08042021-002 Unit ID

93it [00:25,  2.86it/s]

Alfa-14052021-003 Spike ID mismatch: [ 1.  1.  2.  2.  3.  3.  4.  4.  5.  5.  6.  6.  7.  7.  8.  8.  9.  9.
 10. 10. 10. 11. 11. 12. 12. 13. 13. 14. 14. 15. 15. 16. 16. 17. 17. 18.
 18. 19. 19. 20. 20. 21. 21. 22. 22. 23. 23. 24. 24. 24. 25. 25. 26. 26.
 27. 27. 28. 28. 29. 29. 30. 30. 31. 31. 32. 32. 32. 33. 33. 34. 34. 35.
 35. 35. 36. 36. 37. 37. 38. 38. 39. 39. 40. 40. 41. 41. 41. 42. 42. 43.
 43. 44. 44. 45. 45. 45. 46. 46. 47. 47. 48. 48. 49. 49. 50. 50. 51. 51.
 52. 52. 53. 53. 54. 54. 55. 55. 56. 56. 57. 57. 58. 58. 58. 59. 59. 60.
 60. 61. 61. 61. 62. 62. 63. 63. 64. 64.] [len: 136] != [ 1  1  2  2  3  3  4  5  5  6  6  7  7  8  8  9  9 10 10 10 11 12 12 13
 13 14 14 15 15 16 17 18 18 19 19 20 21 21 22 23 23 24 24 25 26 26 27 27
 28 28 29 29 30 30 31 32 32 32 33 33 34 34 35 35 35 36 36 37 37 38 38 39
 39 40 40 41 41 41 42 42 43 43 44 44 45 45 45 46 46 47 47 48 48 49 49 50
 50 51 51 52 52 53 53 54 54 55 55 56 56 57 57 58 58 58 59 59 60 60 61 61
 61 62 62 63 63 64 64] [len: 12

107it [00:31,  3.37it/s]


In [30]:
# data["meta"]['spikeID'].shape
from neuro_data_analysis.PL2_reading_lib import read_channel_unit_info, pl2_info
local_PL2_root = r"S:\Data-Ephys-Raw"
network_PL2_root = r"N:\Data-Ephys-Raw"
for _, exprow in tqdm(pd.concat([ExpRecord_Hessian_All, ExpRecord_Evolution_All]).iterrows()):   
    PL2_path = join(local_PL2_root, f"{exprow.ephysFN}.pl2")
    if not os.path.exists(PL2_path):
        PL2_path = PL2_path.replace("S:", "N:") # use the network drive for non local PL2 files
        if not os.path.exists(PL2_path):
            # print(f"PL2 file does not exist on local or network drive: {PL2_path}")
            continue
    spike_channel_id, spike_unit_id, spike_total_cnt = read_channel_unit_info(PL2_path)
    pkldata = pkl.load(open(join(pkl_root, f"{exprow.ephysFN}.pkl"), "rb"))
    if "spikeID" in pkldata["meta"].keys():
        length_match = len(pkldata["meta"]["spikeID"][0]) == len(spike_channel_id)
        exact_match = (pkldata["meta"]["spikeID"][0].astype(int) == spike_channel_id)
        if not (length_match and exact_match.all()):
            print(f"{exprow.ephysFN} Spike ID mismatch: {pkldata['meta']['spikeID']} [len: {len(pkldata['meta']['spikeID'][0])}] != {spike_channel_id} [len: {len(spike_channel_id)}]")
    # else:
    #     pkldata["meta"]["spikeID"] = spike_channel_id[None, :]
        
    if "unitID" in pkldata["meta"].keys():
        length_match = len(pkldata["meta"]["unitID"][0]) == len(spike_unit_id)
        exact_match = (pkldata["meta"]["unitID"][0].astype(int) == spike_unit_id)
        if not (length_match and exact_match.all()):
            print(f"{exprow.ephysFN} Unit ID mismatch: {pkldata['meta']['unitID']} [len: {len(pkldata['meta']['unitID'][0])}] != {spike_unit_id} [len: {len(spike_unit_id)}]")
    # else:
    #     pkldata["meta"]["unitID"] = spike_unit_id[None, :]
    # pkldata["meta"]["total_spike_cnt"] = spike_total_cnt
    # pkl.dump(pkldata, open(join(pkl_root, f"{exprow.ephysFN}.pkl"), "wb"))



68it [00:06,  9.16it/s]C:\Users\ponce\AppData\Local\Temp\ipykernel_58096\883216326.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  exact_match = (pkldata["meta"]["spikeID"][0].astype(int) == spike_channel_id)
C:\Users\ponce\AppData\Local\Temp\ipykernel_58096\883216326.py:24: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  exact_match = (pkldata["meta"]["unitID"][0].astype(int) == spike_unit_id)
90it [00:06, 24.35it/s]

Alfa-12032021-004 Spike ID mismatch: [[ 1.  1.  2.  2.  3.  3.  4.  5.  5.  5.  6.  6.  7.  8.  8.  8.  9.  9.
  10. 11. 12. 13. 14. 15. 16. 17. 18. 18. 19. 19. 20. 20. 21. 22. 23. 24.
  24. 25. 25. 26. 27. 27. 28. 28. 29. 30. 31. 32. 32. 32. 33. 33. 34. 34.
  34. 35. 35. 36. 37. 37. 38. 39. 39. 40. 40. 41. 41. 42. 42. 43. 43. 43.
  44. 45. 46. 47. 48. 49. 50. 50. 51. 52. 53. 53. 54. 55. 55. 56. 57. 58.
  58. 58. 58. 59. 59. 60. 61. 62. 63. 63. 64. 64.]] [len: 102] != [ 1  1  2  2  3  4  5  5  5  6  7  8  8  8  9  9 10 11 12 13 14 15 16 17
 18 19 20 21 22 23 24 25 26 27 28 28 29 30 31 32 32 32 33 33 34 34 34 35
 35 36 37 37 38 39 39 40 40 41 41 42 42 43 43 43 44 45 46 47 48 49 50 50
 51 52 53 53 54 55 55 56 57 58 58 58 58 59 60 61 62 63 63 64 64] [len: 93]
Alfa-12032021-004 Unit ID mismatch: [[0. 1. 0. 1. 0. 1. 1. 0. 1. 2. 0. 1. 1. 0. 1. 2. 0. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0.
  1. 2. 0. 1. 0. 1. 2. 0. 1. 1. 0. 1. 1. 0. 1. 0.

107it [00:07, 14.09it/s]

Alfa-14052021-003 Spike ID mismatch: [[ 1.  1.  2.  2.  3.  3.  4.  4.  5.  5.  6.  6.  7.  7.  8.  8.  9.  9.
  10. 10. 10. 11. 11. 12. 12. 13. 13. 14. 14. 15. 15. 16. 16. 17. 17. 18.
  18. 19. 19. 20. 20. 21. 21. 22. 22. 23. 23. 24. 24. 24. 25. 25. 26. 26.
  27. 27. 28. 28. 29. 29. 30. 30. 31. 31. 32. 32. 32. 33. 33. 34. 34. 35.
  35. 35. 36. 36. 37. 37. 38. 38. 39. 39. 40. 40. 41. 41. 41. 42. 42. 43.
  43. 44. 44. 45. 45. 45. 46. 46. 47. 47. 48. 48. 49. 49. 50. 50. 51. 51.
  52. 52. 53. 53. 54. 54. 55. 55. 56. 56. 57. 57. 58. 58. 58. 59. 59. 60.
  60. 61. 61. 61. 62. 62. 63. 63. 64. 64.]] [len: 136] != [ 1  1  2  2  3  3  4  5  5  6  6  7  7  8  8  9  9 10 10 10 11 12 12 13
 13 14 14 15 15 16 17 18 18 19 19 20 21 21 22 23 23 24 24 25 26 26 27 27
 28 28 29 29 30 30 31 32 32 32 33 33 34 34 35 35 35 36 36 37 37 38 38 39
 39 40 40 41 41 41 42 42 43 43 44 44 45 45 45 46 46 47 47 48 48 49 49 50
 50 51 51 52 52 53 53 54 54 55 55 56 56 57 57 58 58 58 59 59 60 60 61 61
 61 62 62 63 63 64 64]

In [26]:
print(len(pkldata["meta"]["spikeID"][0]))
print(len(spike_channel_id))


102
93


In [24]:
exprow.ephysFN

'Alfa-12032021-004'

C:\Users\ponce\AppData\Local\Temp\ipykernel_58096\2089072093.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  (pkldata["meta"]["spikeID"][0].astype(int) == spike_channel_id).all()


AttributeError: 'bool' object has no attribute 'all'

In [19]:
pkldata["meta"]["spikeID"]

array([[ 1.,  1.,  2.,  2.,  3.,  3.,  4.,  5.,  5.,  5.,  6.,  6.,  7.,
         8.,  8.,  8.,  9.,  9., 10., 11., 12., 13., 14., 15., 16., 17.,
        18., 18., 19., 19., 20., 20., 21., 22., 23., 24., 24., 25., 25.,
        26., 27., 27., 28., 28., 29., 30., 31., 32., 32., 32., 33., 33.,
        34., 34., 34., 35., 35., 36., 37., 37., 38., 39., 39., 40., 40.,
        41., 41., 42., 42., 43., 43., 43., 44., 45., 46., 47., 48., 49.,
        50., 50., 51., 52., 53., 53., 54., 55., 55., 56., 57., 58., 58.,
        58., 58., 59., 59., 60., 61., 62., 63., 63., 64., 64.]])

In [5]:
for _, exprow in tqdm(pd.concat([ExpRecord_Hessian_All, ExpRecord_Evolution_All]).iterrows()):
    print(exprow.ephysFN, exprow.Animal, "Exp", int(exprow.Expi))
    data = pkl.load(open(join(pkl_root, f"{exprow.ephysFN}.pkl"), "rb"))
    # rasters = data["rasters"]
    meta = data["meta"]
    # Trials = data["Trials"]
    data["meta"]

Caos-12022024-003 Caos Exp 1
Caos-12022024-006 Caos Exp 2
Caos-12042024-003 Caos Exp 3
Caos-12042024-007 Caos Exp 4
Caos-12092024-004 Caos Exp 5
Caos-12102024-003 Caos Exp 6
Caos-12102024-006 Caos Exp 7
Diablito-03122024-003 Diablito Exp 1
Diablito-11122024-003 Diablito Exp 2
Diablito-11122024-005 Diablito Exp 3
Diablito-12122024-004 Diablito Exp 4
Diablito-12122024-006 Diablito Exp 5
Alfa-04082020-003 Alfa Exp 1
Alfa-06082020-004 Alfa Exp 2
Alfa-10082020-004 Alfa Exp 2
Alfa-12082020-004 Alfa Exp 3
Alfa-14082020-003 Alfa Exp 4
Alfa-25082020-007 Alfa Exp 5
Alfa-27082020-003 Alfa Exp 6
Alfa-01092020-004 Alfa Exp 7
Alfa-02092020-004 Alfa Exp 8
Alfa-02092020-006 Alfa Exp 9
Alfa-08092020-004 Alfa Exp 10
Alfa-11092020-003 Alfa Exp 11
Alfa-11092020-006 Alfa Exp 12
Alfa-16092020-003 Alfa Exp 13
Alfa-16092020-005 Alfa Exp 14
Alfa-27102020-007 Alfa Exp 15
Alfa-27102020-008 Alfa Exp 15
Alfa-27102020-009 Alfa Exp 15
Alfa-04112020-005 Alfa Exp 16
Alfa-06112020-006 Alfa Exp 17
Alfa-12012021-003 Alfa

### Dev zone

In [ ]:
stim_info = []
re_pattern = r'(noise|class)_eig(\d+)_lin([+-]?\d+\.\d+)'
for name in unique_imgnames: 
    match = re.match(re_pattern, name)
    if match:
        space_name = match.groups()[0]
        eig_value = int(match.groups()[1])
        lin_value = float(match.groups()[2])
        stim_info.append({"img_name": name, "space_name": space_name, "eig_id": eig_value, "lin_dist": lin_value, "hessian_img": True, "trial_ids": indices_per_name[name]})
    else:
        stim_info.append({"img_name": name, "space_name": None, "eig_id": None, "lin_dist": None, "hessian_img": False, "trial_ids": indices_per_name[name]})

stim_info_df = pd.DataFrame(stim_info)


In [ ]:
from tqdm import tqdm
from neuro_data_analysis.mat_data_translate_lib import h5_to_dict_simplify, print_hdf5_info
mat_root = r"S:\Data-Ephys-MAT"
pkl_root = r"S:\Data-Ephys-PKL"
T0 = time.time()
for ephys_name in tqdm(ExpRecord_Hessian_All.ephysFN.values):
    if os.path.exists(join(pkl_root, f"{ephys_name}.pkl")):
        print(f"PKL file already exists for {ephys_name}")
        continue
    t0 = time.time()
    mat_file = join(mat_root, f"{ephys_name}_formatted.mat")
    data = h5py.File(mat_file, "r")
    print(f"Loading {ephys_name} mat file")
    t1 = time.time()
    print(f"Time taken: {t1 - t0:.2f} seconds for loading mat file")
    # fast selective loading
    meta_dict = h5_to_dict_simplify(data['meta'], data)
    # turn rasters into a numpy array, reduce precision to 32-bit float
    rasters = np.array(data['rasters']).astype(np.float32)
    # lfps = np.array(data['lfps']).astype(np.float32)
    Trial_dict = edict()
    for key in data['Trials'].keys():
        if key in ["B", "MLConfig"]:
            continue
        print(f"Loading {key}", end="\t")
        Trial_dict[key] = h5_to_dict_simplify(data['Trials'][key], data)
        print("done")
    # TrialRecord_dict = h5_to_dict_simplify(data['Trials/TrialRecord'], data)
    t2 = time.time()
    print(f"Time taken: {t2 - t1:.2f} seconds for loading trials")

    pkl.dump({"Trials": Trial_dict, "meta": meta_dict, "rasters": rasters, }, open(join(pkl_root, f"{ephys_name}.pkl"), "wb"))
    t3 = time.time()
    print(f"Time taken: {t3 - t2:.2f} seconds for dumping pkl file [Total: {t3 - T0:.2f} seconds]")


  0%|          | 0/41 [00:00<?, ?it/s]

PKL file already exists for Caos-12022024-003
PKL file already exists for Caos-12022024-006
PKL file already exists for Caos-12042024-003
PKL file already exists for Caos-12042024-007
PKL file already exists for Caos-12092024-004
PKL file already exists for Caos-12102024-003
PKL file already exists for Caos-12102024-006
PKL file already exists for Diablito-03122024-003
PKL file already exists for Diablito-11122024-003
PKL file already exists for Diablito-11122024-005
PKL file already exists for Diablito-12122024-004
PKL file already exists for Diablito-12122024-006
Loading Alfa-04082020-003 mat file
Time taken: 0.33 seconds for loading mat file
Loading TrialRecord	Warning: No MATLAB class specified for <HDF5 dataset "Angle": shape (5, 1), type "|O"> at path /Trials/TrialRecord/CurrentConditionStimulusInfo/Angle object
done
Loading XY	done
Loading block	done
Loading event03	done
Loading event10	done
Loading eventMarkers	done
Loading eyePupil	done
Loading eyeXY	done
Loading imageName	don

 32%|███▏      | 13/41 [00:08<00:17,  1.61it/s]

Time taken: 0.51 seconds for dumping pkl file [Total: 8.12 seconds]
Loading Alfa-06082020-004 mat file
Time taken: 0.08 seconds for loading mat file
Loading TrialRecord	Warning: No MATLAB class specified for <HDF5 dataset "Angle": shape (5, 1), type "|O"> at path /Trials/TrialRecord/CurrentConditionStimulusInfo/Angle object
done
Loading XY	done
Loading block	done
Loading event03	done
Loading event10	done
Loading eventMarkers	done
Loading eyePupil	done
Loading eyeXY	done
Loading imageName	done
Loading trialStart	done
Loading width	done
Loading words	done
Time taken: 7.62 seconds for loading trials


 34%|███▍      | 14/41 [00:16<00:36,  1.36s/it]

...

Time taken: 0.73 seconds for dumping pkl file [Total: 319.99 seconds]
Loading Beto-14092020-004 mat file
Time taken: 0.06 seconds for loading mat file
Loading TrialRecord	Warning: No MATLAB class specified for <HDF5 dataset "Angle": shape (6, 1), type "|O"> at path /Trials/TrialRecord/CurrentConditionStimulusInfo/Angle object
done
Loading XY	done
Loading block	done
Loading event03	done
Loading event10	done
Loading eventMarkers	done
Loading eyePupil	done
Loading eyeXY	done
Loading imageName	done
Loading trialStart	done
Loading width	done
Loading words	done
Time taken: 9.65 seconds for loading trials


 98%|█████████▊| 40/41 [05:30<00:11, 11.69s/it]

Time taken: 1.06 seconds for dumping pkl file [Total: 330.77 seconds]
Loading Beto-17092020-003 mat file
Time taken: 0.07 seconds for loading mat file
Loading TrialRecord	Warning: No MATLAB class specified for <HDF5 dataset "Angle": shape (5, 1), type "|O"> at path /Trials/TrialRecord/CurrentConditionStimulusInfo/Angle object
done
Loading XY	done
Loading block	done
Loading event03	done
Loading event10	done
Loading eventMarkers	done
Loading eyePupil	done
Loading eyeXY	done
Loading imageName	done
Loading trialStart	done
Loading width	done
Loading words	done
Time taken: 11.91 seconds for loading trials


100%|██████████| 41/41 [05:43<00:00,  8.38s/it]

Time taken: 0.76 seconds for dumping pkl file [Total: 343.52 seconds]


In [ ]:
for _, exprow in ExpRecord_Evolution_All.iterrows():
    print(exprow.ephysFN, exprow.Animal, "Exp", int(exprow.Expi))
    data = pkl.load(open(join(pkl_root, f"{exprow.ephysFN}.pkl"), "rb"))
    rasters = data["rasters"]
    meta = data["meta"]
    Trials = data["Trials"]
    # imageName = np.squeeze(Trials.imageName)
    stimuli_dir = exprow.stimuli
    prefchan = np.squeeze(Trials.TrialRecord.User.prefChan)
    prefunit = np.squeeze(Trials.TrialRecord.User.unit_perChan)
    imgpos = np.squeeze(Trials.TrialRecord.User.evoConfiguration[1])
    imgwidth = np.squeeze(Trials.TrialRecord.User.evoConfiguration[2])
    print(int(prefchan[0]), int(prefunit[0]), imgpos[0].T, imgwidth[0].item())
    # find the corresponding row in ExpRecord_Hessian_All
    hessian_exprow = ExpRecord_Hessian_All.query(f"Animal == '{exprow.Animal}' and Expi == {exprow.Expi}")
    # print(hessian_exprow)
    assert (hessian_exprow.pref_chan == prefchan[0]).all(), f"Pref chan mismatch: {hessian_exprow.pref_chan} != {prefchan[0]}"
    assert (hessian_exprow.pref_unit == prefunit[0]).all(), f"Pref unit mismatch: {hessian_exprow.pref_unit} != {prefunit[0]}"
    assert (hessian_exprow.stim_size == imgwidth[0].item()).all(), f"Stim size mismatch: {hessian_exprow.stim_size} != {imgwidth[0]}"


Caos-12022024-002 Caos Exp 1
91 2 [[0. 0.]] 2.0


AssertionError: Stim size mismatch: 12   NaN
Name: stim_size, dtype: float64 != [[2.]]

In [ ]:
Trials.TrialRecord.User.evoConfiguration

array([[array([[29.]]), array([[29.]])],
       [array([[0.],
               [1.]]), array([[0.],
                              [1.]])],
       [array([[3.]]), array([[3.]])],
       [array([[1.]]), array([[1.]])],
       ['CMAES', 'CMAES']], dtype=object)

In [ ]:
imgpos = Trials.TrialRecord.User.evoConfiguration[1]
imgwidth = Trials.TrialRecord.User.evoConfiguration[2]
print(imgpos, imgwidth)

[array([[0.],
        [1.]]) array([[0.],
                      [1.]])] [array([[3.]]) array([[3.]])]
